Iteradores:

Un iterador en Python es un objeto que se utiliza para iterar sobre elementos iterables (como listas, tuplas, diccionarios y conjuntos). Un iterador
implementa dos métodos esenciales:iter_() y _next_()

__iter_() devuelve el objeto iterador mismo y se utiliza una vez al principio del ciclo.
__next_() devuelve el siguiente elemento de la secuencia. Cuando no hay más elementos, lanza una excepción Stoplteration.

__init__ es el constructor de la clase inicia, se utiliza para inicializar los atributos de la clase.

=== Empleados de Ventas ===
Ana (ID: 1)
Carlos (ID: 3)

¿Por qué usar esto en análisis de datos?
✅ Eficiencia: Procesas datos en lotes sin cargar todo en memoria.
✅ Reutilizable: La lógica de filtrado/transformación se encapsula en una clase.
✅ Legibilidad: El código queda más claro que con for anidados + if.

Bonus: ¿Cómo mejorarías estos iteradores?
🔹 Añadir manejo de errores (ej: si un campo no existe).
🔹 Permitir múltiples filtros (ej: edad + departamento).
🔹 Implementar __len__() para saber cuántos elementos quedan.


In [1]:
class Contador:
    def __init__(self,bajo ,alto):
        self.actual = bajo
        self.alto = alto
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.actual <= self.alto:
            num = self.actual
            self.actual += 1
            return num
        raise StopIteration
    
for num in Contador (3,8):
    print(num)    

3
4
5
6
7
8


In [ ]:
class IteradorDepartamento:
    def __init__(self, empleados, departamento):
        self.empleados = empleados
        self.departamento = departamento
        self.indice = 0

    def __iter__(self):
        return self  # Devuelve el propio objeto como iterador

    def __next__(self):
        while self.indice < len(self.empleados):
            empleado = self.empleados[self.indice]
            self.indice += 1
            if empleado["departamento"] == self.departamento:
                return empleado
        raise StopIteration  # Termina la iteración

# Datos de ejemplo
empleados = [
    {"id": 1, "nombre": "Ana", "departamento": "Ventas"},
    {"id": 2, "nombre": "Luis", "departamento": "IT"},
    {"id": 3, "nombre": "Carlos", "departamento": "Ventas"},
    # ... (otros 7 registros)
]

# Uso:
print("=== Empleados de Ventas ===")
for emp in IteradorDepartamento(empleados, "Ventas"):
    print(f"{emp['nombre']} (ID: {emp['id']})")

    

In [ ]:
"""Ejemplo 2: Iterador para Paginar Resultados (Lotes de 3 Registros)
Objetivo: Procesar los datos en lotes (ej: para evitar cargar toda la base a la vez).

=== Procesando en lotes de 3 ===
Lote 1: ['Ana', 'Luis', 'Carlos']
Lote 2: [...]
..."""
class IteradorPorLotes:
    def __init__(self, data, tamano_lote=3):
        self.data = data
        self.tamano_lote = tamano_lote
        self.indice = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.indice >= len(self.data):
            raise StopIteration
        lote = self.data[self.indice : self.indice + self.tamano_lote]
        self.indice += self.tamano_lote
        return lote

# Uso:
print("=== Procesando en lotes de 3 ===")
for i, lote in enumerate(IteradorPorLotes(empleados), 1):
    print(f"Lote {i}: {[emp['nombre'] for emp in lote]}")

In [ ]:
"""Ejemplo 3: Iterador para Validar Datos (Edad > 30)
Objetivo: Recorrer solo empleados que cumplan una condición (ej: mayores de 30 años).
=== Empleados mayores de 30 años ===
Luis (32 años)
Carlos (45 años)"""
class IteradorFiltroEdad:
    def __init__(self, empleados, edad_minima):
        self.empleados = empleados
        self.edad_minima = edad_minima
        self.indice = 0

    def __iter__(self):
        return self

    def __next__(self):
        while self.indice < len(self.empleados):
            empleado = self.empleados[self.indice]
            self.indice += 1
            if empleado["edad"] and empleado["edad"] > self.edad_minima:
                return empleado
        raise StopIteration

# Uso:
print("=== Empleados mayores de 30 años ===")
for emp in IteradorFiltroEdad(empleados, 30):
    print(f"{emp['nombre']} ({emp['edad']} años)")

In [ ]:
"""Ejemplo 4: Iterador para Proyectar Campos (Solo Nombres)
Objetivo: Extraer solo un campo específico (ej: lista de nombres).

['Ana', 'Luis', 'Carlos', ...]"""

class IteradorNombres:
    def __init__(self, empleados):
        self.empleados = empleados
        self.indice = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.indice < len(self.empleados):
            nombre = self.empleados[self.indice]["nombre"]
            self.indice += 1
            return nombre
        raise StopIteration

# Uso:
print("=== Lista de Nombres ===")
nombres = list(IteradorNombres(empleados))  # Convertir a lista
print(nombres)

Funciones Generadoras:

Una función generadora es una foma simple de crear un iterador. Se define como una función regular, pero utiliza la declaración yield para
devolver datos. Una vez que una funcion generadora produce un valor (con yield), la funcion se pausa y la ejecucion se transfiere al llamador. La
función puede luego continuar desde donde se detuvo después de la última llamada de yield .

In [4]:
def contador ( bajo, alto):
    while bajo <= alto:
        yield bajo
        bajo += 1

for i in contador (3,8):
    print(i)

3
4
5
6
7
8


Explicación:
LimiteExcedidoError:

Excepción personalizada que se lanza si el limite pedido es mayor que la cantidad de números disponibles en el rango (alto - bajo + 1).

contador(bajo, alto, limite=None):

Si limite es None, genera todos los números en el rango (como tu versión original).

Si limite es mayor que los números disponibles, lanza LimiteExcedidoError.

Usa un contador interno (count) para detenerse al alcanzar el limite.

¿Por qué es útil para análisis de datos?
Control de memoria: Limitar la cantidad de datos generados evita sobrecargar la RAM.

Validación temprana: La excepción evita procesamientos innecesarios si el límite no es válido.

Reutilizable: Puedes integrarlo en pipelines de datos para procesar chunks.

In [ ]:
class LimiteExcedidoError(Exception):
    """Se lanza cuando se piden más elementos de los disponibles en el rango"""
    def __init__(self, cantidad_solicitada, cantidad_disponible):
        mensaje = f"⚠️ Error: Quisiste {cantidad_solicitada} elementos, pero solo hay {cantidad_disponible} en el rango."
        super().__init__(mensaje)

def contador(bajo, alto, limite=None):
    """
    Genera números desde 'bajo' hasta 'alto' (inclusive), 
    pero limita la cantidad si se especifica 'limite'.
    """
    disponibles = alto - bajo + 1
    if limite is not None and limite > disponibles:
        raise LimiteExcedidoError(limite, disponibles)
    
    count = 0
    while bajo <= alto and (limite is None or count < limite):
        yield bajo
        bajo += 1
        count += 1

# --- Ejemplos de uso ---
print("=== Ejemplo 1: Sin límite ===")
for num in contador(3, 8):  # Genera todos los números (3, 4, 5, 6, 7, 8)
    print(num)

print("\n=== Ejemplo 2: Con límite (3 elementos) ===")
for num in contador(3, 8, limite=3):  # Genera solo 3, 4, 5
    print(num)

print("\n=== Ejemplo 3: Límite excedido (error) ===")
try:
    for num in contador(3, 8, limite=10):  # Error: solo hay 6 números disponibles
        print(num)
except LimiteExcedidoError as e:
    print(e)